# Label CT Scan cracks

This notebook is used to label CT Scan crack

## Open Image and (possibly) pre-existing label

In [24]:
from skimage.io import imread, imsave
import napari
import os
import numpy as np
from pathlib import Path
from tnia.deeplearning.dl_helper import make_label_directory

tnia_images_path = Path(r'D:/images')
parent_path=Path(tnia_images_path / r'tnia-python-images/imagesc/2024_05_07_ct_scan')
label_path = Path(parent_path / r'labels')
image_path = Path(parent_path / r'images')
name = 'Cored_NonMachined_rec00000489'
ext = '.bmp'

image_label_paths, mask_label_paths = make_label_directory(1, 1, label_path)

for mask_label_name in image_label_paths:
    print(mask_label_name)
print()

im = imread(os.path.join(image_path / (name+ext)))

# this image is really gray scale with each channel containing the same info so keep only the first channel
im = im[:,:,0]

try:
    label = imread(os.path.join(mask_label_path / (name+".tif")))
except:
    print('labels not created yet', (os.path.join(label_path / (name+".tif"))))
    label = np.zeros([im.shape[0], im.shape[1]], dtype=np.uint8)


D:\images\tnia-python-images\imagesc\2024_05_07_ct_scan\labels\input0

labels not created yet D:\images\tnia-python-images\imagesc\2024_05_07_ct_scan\labels\Cored_NonMachined_rec00000489.tif


## Edit an ROI of the labels in Napari

Now edit the labels in Napari.  We can start from a blank image or alternatively a pre-existing set of labels (either generated by another method, or previous manual labels) that we want to improve.

Use the box ROI to indicate which region of the image contains labels.

In [25]:
viewer = napari.Viewer()
viewer.add_image(im, name='im')
viewer.add_labels(label, name='labels')

boxes_layer = viewer.add_shapes(
            name="Label box",
            face_color="transparent",
            edge_color="green",
            edge_width=2,
        )

boxes_layer.add([[[0,0],[im.shape[0],im.shape[1]//2]]])

## Get the ROI that was labeled

In [26]:
ystart = int(np.min(boxes_layer.data[0][:,0]))
yend = int(np.max(boxes_layer.data[0][:,0]))
xstart = int(np.min(boxes_layer.data[0][:,1]))
xend = int(np.max(boxes_layer.data[0][:,1]))


ystart = max(0, ystart)
yend = min(im.shape[0], yend)
xstart = max(0, xstart)
xend = min(im.shape[1], xend)

print('bounding box is',ystart, yend, xstart, xend)
if np.ndim(im) == 3:
    im = im[ystart:yend, xstart:xend, :]
else:
    im = im[ystart:yend, xstart:xend]

label = label[ystart:yend, xstart:xend]

bounding box is 1272 1791 951 1619


In [27]:
im.shape

(519, 668)

## Save modified labels

In [28]:
mask_label_paths

['D:\\images\\tnia-python-images\\imagesc\\2024_05_07_ct_scan\\labels\\ground truth0']

## Get next label names

In this step we generate label names.  It should be based on the input file name, however if a label from that image already exists the label number should be incremented so we don't overwrite it. 

In [29]:
from tnia.deeplearning.dl_helper import generate_label_names, generate_next_label_name

image_name, mask_name = generate_label_names(str(image_label_paths[0]), str(mask_label_paths[0]), name)

print(image_name)
print(mask_name)



D:\images\tnia-python-images\imagesc\2024_05_07_ct_scan\labels\input0/Cored_NonMachined_rec00000489_1.tif
D:\images\tnia-python-images\imagesc\2024_05_07_ct_scan\labels\ground truth0/Cored_NonMachined_rec00000489_1.tif


In [30]:
imsave(image_name, im)
imsave(mask_name, label)

C:\Users\bnort\AppData\Local\Temp\ipykernel_13364\1705855600.py:2: UserWarning: D:\images\tnia-python-images\imagesc\2024_05_07_ct_scan\labels\ground truth0/Cored_NonMachined_rec00000489_1.tif is a low contrast image
  imsave(mask_name, label)


In [31]:
print(image_name)
json_name = os.path.basename(image_name).split('.')[0]+'.json'
print(json_name)

D:\images\tnia-python-images\imagesc\2024_05_07_ct_scan\labels\input0/Cored_NonMachined_rec00000489_1.tif
Cored_NonMachined_rec00000489_1.json


In [32]:
# take away extension from image_name and replace with json
json_name = os.path.basename(image_name).split('.')[0]+'.json'

# save xstart, ystart, xend, yend to json 
import json
json_name = os.path.join(image_label_paths[0], json_name)
with open(json_name, 'w') as f:
    json_ = {}
    json_['source_name'] = name + ext
    json_['bbox'] = [xstart, ystart, xend, yend]
    json.dump(json_, f)

In [33]:
print(json_name)

D:\images\tnia-python-images\imagesc\2024_05_07_ct_scan\labels\input0\Cored_NonMachined_rec00000489_1.json


In [34]:
viewer = napari.Viewer()
viewer.show()